En este archivo voy a desarrollar los prototipos necesarios que me permitan desplegar el sistema de recomendación user-item based, tanto en local cómo en Render.
Se me pide lo siguiente:
![alt text](../img/sistema_recomendacion_user_item.png "Title")

Primer prototipo (voy a copiar y pegar todo el código previamente creado, para a continuación borrar las partes innecesarias)

In [1]:
# Para procesar los datos
import pandas as pd
import numpy as np
import scipy.stats as stats

# Para visualizar los datos
import seaborn as sns

# Para la similitud
from sklearn.metrics.pairwise import cosine_similarity

juegos = pd.read_parquet("../Datasets/steam_games_complete.parquet")
reseñas = pd.read_parquet("../Datasets/reviews_con_puntaje.parquet")

juegos_con_mas_de_100_reseñas = reseñas["item_id"].value_counts() > 100 # 99 juegos con más de 100 reseñas

# Lista que me va a servir para eliminar los juegos que no tengan más de 100 reseñas
lista_booleana_para_mascara = []
for id in reseñas["item_id"]: # Por cada id en la columna de item_id de las reseñas,
    lista_booleana_para_mascara.append(juegos_con_mas_de_100_reseñas[id]) # Se agrega el valor correspondiente a la lista booleana, si tiene más de 100 reseñas, se agrega True, sino, false. Esto lo hago porque la variable "juegos_con_mas_de_100_reseñas" sólo tiene en cuenta valores únicos, y para aplicar una mascara, yo necesito saber en cada fila si corresponde con True o False

reseñas = reseñas[lista_booleana_para_mascara] # Aplico la mascara previamente creada

usuarios_con_mas_de_5_reseñas = reseñas["user_id"].value_counts() > 5 # Creo una variable que almacene con True los usuarios que hicieron más de 5 reseñas, y con False los que hicieron 5 reseñas o menos.

# Aplico el mismo procedimiento que con los juegos
lista_booleana_para_mascara = []
for id in reseñas["user_id"]:
    lista_booleana_para_mascara.append(usuarios_con_mas_de_5_reseñas[id])

reseñas = reseñas[lista_booleana_para_mascara] # Aplico la máscara para dejar unicamente a los usuarios con más de 5 reseñas

reseñas["puntaje_aumentado"] =  reseñas["puntaje"] + 1 # Lo que voy a hacer es aumentar el valor del puntaje de las reseñas en 1, considerando las negativas con un 1, las neutras con un 2 y las positivas con un 3.

print(len(reseñas["item_id"].unique())) # 99 juegos
print(len(reseñas["user_id"].unique())) # 456 usuarios

juegos_y_reseñas = pd.merge(reseñas,juegos,on="item_id") # Con esto lo que busco hacer es unir las tablas tanto de juegos cómo la de reseñas, porque en la de reseñas no tengo el nombre del juego, y es un valor útil.

# A continuación, elimino con un bucle las columnas innecesarias
for columna in juegos_y_reseñas.columns:
    if columna not in ["user_id","puntaje_aumentado","item_name"]:
        juegos_y_reseñas.drop(columna, inplace=True, axis=1)
print(juegos_y_reseñas.columns)

# Creo una nueva tabla que cómo nombre de columnas va a tener el nombre de los juegos, cómo indice de las filas va a tener el ID de cada usuario y cómo valores va a tener el puntaje
matrix = juegos_y_reseñas.pivot_table(index='user_id',columns='item_name',values='puntaje_aumentado')

# Normalizo la tabla previamente creada restandole la media de cada fila a la fila correspondiente.
matrix_norm = matrix.subtract(matrix.mean(axis=1), axis = 'rows')

# Rellenar NaN con 0s ya que el coseno no se ve afectado por ceros
matrix_norm_filled = matrix_norm.fillna(0)

# Calcular la similitud del coseno
similitud_del_coseno = cosine_similarity(matrix_norm_filled)

similitud_de_usuarios_coseno = pd.DataFrame(similitud_del_coseno, index=matrix_norm.index, columns=matrix_norm.index)

99
456
Index(['user_id', 'puntaje_aumentado', 'item_name'], dtype='object')


Bueno, teniendo en consideración todo lo que acabo de hacer, las tablas que necesito son:

- similitud_de_usuarios_coseno
- matrix_norm

Asi que procedo a exportar en formato parquet las tablas correspondientes a esas variables para poderme manejar de forma más cómoda en la función

In [2]:
similitud_de_usuarios_coseno.to_parquet("../Datasets/recomendacion/similitud_entre_usuarios.parquet",compression="snappy")
similitud_de_usuarios_coseno

user_id,09879655452567,10051997,1011001,111222333444555666888,1234567io9872345678765432,1873410337,1snap,210396,29123,2sBs,...,xtomx_freedom,yotuic,you_re_ded,youngbenaffleck,zaaikbr,zakbot,zayyntt,zerzang,zrustz16,zyr0n1c
user_id,,,,,,,,,,,,,,,,,,,,,
09879655452567,1.000000,0.000000,0.000000,0.000000,0.000000,0.111803,0.265334,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.167705,0.000000,0.0,0.000000,0.0
10051997,0.000000,1.000000,-0.154746,-0.333333,-0.730297,0.821584,0.000000,0.117851,0.288675,0.484481,...,0.521749,-0.125988,0.195180,0.182574,0.000000,0.000000,-0.091287,0.0,-0.596285,0.0
1011001,0.000000,-0.154746,1.000000,0.154746,0.355983,-0.339032,0.000000,0.109422,0.268028,-0.034602,...,0.069205,0.058489,0.181220,-0.288177,0.346023,0.000000,-0.084758,0.0,0.207614,0.0
111222333444555666888,0.000000,-0.333333,0.154746,1.000000,0.365148,-0.365148,0.000000,0.000000,0.000000,-0.186339,...,-0.149071,0.062994,0.000000,-0.091287,0.000000,0.000000,0.000000,0.0,0.223607,0.0
1234567io9872345678765432,0.000000,-0.730297,0.355983,0.365148,1.000000,-0.800000,0.000000,0.000000,0.000000,-0.306186,...,-0.244949,0.345033,-0.106904,-0.275000,0.000000,0.000000,0.050000,0.0,0.571548,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zakbot,-0.167705,0.000000,0.000000,0.000000,0.000000,-0.075000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.262500,0.000000,1.000000,0.000000,0.0,0.000000,0.0
zayyntt,0.000000,-0.091287,-0.084758,0.000000,0.050000,0.000000,0.339032,-0.064550,-0.158114,0.183712,...,-0.122474,0.207020,-0.213809,0.000000,-0.163299,0.000000,1.000000,0.0,0.081650,0.0
zerzang,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [3]:
matrix_norm.to_parquet("../Datasets/recomendacion/matriz_normalizada.parquet",compression="snappy")
matrix_norm

item_name,APB Reloaded,ARK: Survival Evolved,Ace of Spades: Battle Builder,AdVenture Capitalist,Arma 3,Awesomenauts - the 2D moba,Bad Rats: the Rats' Revenge,BattleBlock Theater®,BioShock Infinite,Blacklight: Retribution,...,The Stanley Parable,The Walking Dead,The Walking Dead: Season 2,Tomb Raider,Trove,Undertale,Unturned,Verdun,Warframe,XCOM: Enemy Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
09879655452567,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN
10051997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111222333444555666888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.666667,0.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234567io9872345678765432,-0.2,NaN,NaN,NaN,-0.2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zakbot,NaN,-0.6,-0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zayyntt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.2,NaN,NaN,NaN
zerzang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Primer prototipo:

In [4]:
from fastapi import FastAPI, HTTPException, Query

def recomendacion_usuario(usuario_elegido):
  # Lectura de los archivos necesarios para trabajar
  similitud_de_usuarios_coseno = pd.read_parquet("../Datasets/recomendacion/similitud_entre_usuarios.parquet")
  matrix_norm = pd.read_parquet("../Datasets/recomendacion/matriz_normalizada.parquet")

  if usuario_elegido not in matrix_norm.index:
        raise HTTPException(status_code=404, detail="Usuario no encontrado, intente con otro")
  

  # Elijo una determinada cantidad de usuarios similares para encontrar, en mi caso voy a hacer el top 9 de usuarios más similares
  cantidad_de_usuarios_similares = 9
  # El porcentaje minimo de similitud lo establecí en un 36%
  similitud_minima_entre_usuarios = 0.36

  # Busco los usuarios similares y los ordeno desde el más similar al menos similar
  usuarios_similares = similitud_de_usuarios_coseno[similitud_de_usuarios_coseno[usuario_elegido] > similitud_minima_entre_usuarios][usuario_elegido].sort_values(ascending=False)[:cantidad_de_usuarios_similares]

  juegos_jugados_por_usuario_elegido = matrix_norm[matrix_norm.index == usuario_elegido].dropna(axis=1,how="all") # Creamos una nueva tabla que contenga los nombres de los juegos que el usuario elegio ha jugado

  # Juegos que usuarios similares han jugado. Eliminamos los juegos que ninguno de los usuarios similares haya jugado
  juegos_de_usuarios_similares = matrix_norm[matrix_norm.index.isin(usuarios_similares.index)].dropna(axis=1, how='all')

  # Eliminamos los juegos jugados por el usuario elegido
  juegos_de_usuarios_similares.drop(juegos_jugados_por_usuario_elegido.columns,axis=1, inplace=True, errors='ignore')

  # Creamos un diccionario para guardar las puntuaciones promedio de los juegos
  puntaje_promedio = {}

  # Iteramos sobre los juegos de los usuarios similares (que el usuario elegido no jugó)
  for juego in juegos_de_usuarios_similares.columns:
    # Obtenemos el puntaje para el juego sobre el cuál se esté iterando
    puntaje_juego = juegos_de_usuarios_similares[juego]

    # Create a variable to store the score Variable para guardar el puntaje total del juego
    total = 0

    # Variable para contar la cantidad de puntajes que tiene el juego dado
    cantidad = 0

    # Iteramos entre los usuarios similares
    for usuario in usuarios_similares.index:

      # Si el juego fué punteado por el usuario
      if pd.isna(puntaje_juego[usuario]) == False:

        # puntaje_parcial es la multiplicación entre el puntaje entre ambos usuarios (el elegido y el iterado) y entre el puntaje que el usuario iterado le dió al juego en cuestión
        puntaje_parcial = usuarios_similares[usuario] * puntaje_juego[usuario]

        # Agregamos el puntaje parcial al total previamente creado
        total += puntaje_parcial

        # Agregamos 1 a la cantidad
        cantidad += 1
    
    # Luego de iterar sobre cada usuario, obtenemos el puntaje promedio para el juego iterado. El juego se almacena cómo clave dentro del diccionario y el valor es el promedio
    puntaje_promedio[juego] = total / cantidad

  # Convertimos el diccionario a un dataframe
  item_score = pd.DataFrame(puntaje_promedio.items(), columns=['item_name', 'item_score'])

  # Ordenamos los juegos por su puntaje
  ranked_item_score = item_score.sort_values(by='item_score', ascending=False)

  # Mostramos el top m juegos
  m = 5
  return ranked_item_score.head(m)

In [5]:
recomendacion_usuario("09879655452567") # Funciona, pero necesita ciertas modificaciones para estar óptima

,item_name,item_score
3,Counter-Strike: Source,0.333712
11,Sid Meier's Civilization® V,0.333712
0,Arma 3,0.288675
15,Unturned,0.132260
14,Terraria,0.132260


Segundo prototipo

In [6]:
from fastapi import FastAPI, HTTPException, Query

def recomendacion_usuario(usuario_elegido):
  
  """En este segundo prototipo, voy a eliminar las variables innecesarias dentro de la función
  """

  # Lectura de los archivos necesarios para trabajar
  similitud_de_usuarios_coseno = pd.read_parquet("../Datasets/recomendacion/similitud_entre_usuarios.parquet")
  matrix_norm = pd.read_parquet("../Datasets/recomendacion/matriz_normalizada.parquet")

  if usuario_elegido not in matrix_norm.index:
        raise HTTPException(status_code=404, detail="Usuario no encontrado, intente con otro")
  
  # Busco los usuarios similares y los ordeno desde el más similar al menos similar. El 0.3 hace referencia a que el porcentaje minimo de similitud es un 30% y el 9 hace referencia al top 9 de usuarios más similares.
  usuarios_similares = similitud_de_usuarios_coseno[similitud_de_usuarios_coseno[usuario_elegido] > 0.3][usuario_elegido].sort_values(ascending=False)[:9] 

  juegos_jugados_por_usuario_elegido = matrix_norm[matrix_norm.index == usuario_elegido].dropna(axis=1,how="all").columns # Creamos un nuevo array que contenga los nombres de los juegos que el usuario elegio ha jugado

  # Juegos que usuarios similares han jugado. Eliminamos los juegos que ninguno de los usuarios similares haya jugado
  juegos_de_usuarios_similares = matrix_norm[matrix_norm.index.isin(usuarios_similares.index)].dropna(axis=1, how='all')

  # Eliminamos los juegos jugados por el usuario elegido
  juegos_de_usuarios_similares.drop(juegos_jugados_por_usuario_elegido,axis=1, inplace=True, errors='ignore')

  # Creamos un diccionario para guardar las puntuaciones promedio de los juegos
  puntaje_promedio = {}

  # Iteramos sobre los juegos de los usuarios similares (que el usuario elegido no jugó)
  for juego in juegos_de_usuarios_similares.columns:
    # Obtenemos el puntaje para el juego sobre el cuál se esté iterando
    puntaje_juego = juegos_de_usuarios_similares[juego]

    # Create a variable to store the score Variable para guardar el puntaje total del juego
    total = 0

    # Variable para contar la cantidad de puntajes que tiene el juego dado
    cantidad = 0

    # Iteramos entre los usuarios similares
    for usuario in usuarios_similares.index:

      # Si el juego fué punteado por el usuario
      if pd.isna(puntaje_juego[usuario]) == False:

        # puntaje_parcial es la multiplicación entre el puntaje entre ambos usuarios (el elegido y el iterado) y entre el puntaje que el usuario iterado le dió al juego en cuestión
        puntaje_parcial = usuarios_similares[usuario] * puntaje_juego[usuario]

        # Agregamos el puntaje parcial al total previamente creado
        total += puntaje_parcial

        # Agregamos 1 a la cantidad
        cantidad += 1
    
    # Luego de iterar sobre cada usuario, obtenemos el puntaje promedio para el juego iterado. El juego se almacena cómo clave dentro del diccionario y el valor es el promedio
    puntaje_promedio[juego] = total / cantidad

  # Convertimos el diccionario a un dataframe
  item_score = pd.DataFrame(puntaje_promedio.items(), columns=['item_name', 'item_score'])

  # Ordenamos los juegos por su puntaje
  ranked_item_score = item_score.sort_values(by='item_score', ascending=False)

  # Mostramos el top m juegos
  m = 5
  lista_salida = []
  for juego in ranked_item_score["item_name"]:
     lista_salida.append(juego)
     if len(lista_salida) == 5:
        break
  return lista_salida

In [7]:
recomendacion_usuario("09879655452567") # Con estos cambios ya tecnicamente cumplí con lo que me pide la consigna para este sistema de recomendación, pero considero que se puede seguir optimizando.

['Counter-Strike: Source',
 "Sid Meier's Civilization® V",
 'Arma 3',
 'Terraria',
 'Starbound']

Tercer prototipo

In [8]:
def recomendacion_usuario(usuario_elegido):
  
  """En este tercer prototipo, voy a continuar optimizando la función
  """

  # Lectura de los archivos necesarios para trabajar
  similitud_de_usuarios_coseno = pd.read_parquet("../Datasets/recomendacion/similitud_entre_usuarios.parquet")
  matrix_norm = pd.read_parquet("../Datasets/recomendacion/matriz_normalizada.parquet")

  if usuario_elegido not in matrix_norm.index:
        raise HTTPException(status_code=404, detail="Usuario no encontrado, intente con otro")
  
  # Busco los usuarios similares y los ordeno desde el más similar al menos similar. El 0.3 hace referencia a que el porcentaje minimo de similitud es un 30% y el 9 hace referencia al top 9 de usuarios más similares.
  usuarios_similares = similitud_de_usuarios_coseno[similitud_de_usuarios_coseno[usuario_elegido] > 0.3][usuario_elegido].sort_values(ascending=False)[:9] 

  juegos_jugados_por_usuario_elegido = matrix_norm[matrix_norm.index == usuario_elegido].dropna(axis=1,how="all").columns # Creamos un nuevo array que contenga los nombres de los juegos que el usuario elegio ha jugado

  # Juegos que usuarios similares han jugado. Eliminamos los juegos que ninguno de los usuarios similares haya jugado
  juegos_de_usuarios_similares = matrix_norm[matrix_norm.index.isin(usuarios_similares.index)].dropna(axis=1, how='all')

  # Eliminamos los juegos jugados por el usuario elegido
  juegos_de_usuarios_similares.drop(juegos_jugados_por_usuario_elegido,axis=1, inplace=True, errors='ignore')

  # Multiplicamos las puntuaciones de los juegos por las similitudes de los usuarios
  puntajes_ponderados = juegos_de_usuarios_similares.mul(usuarios_similares, axis=0)

    # Sumamos los puntajes ponderados por juego, ignorando NaN
  suma_puntajes = puntajes_ponderados.sum(axis=0, skipna=True)

    # Contamos la cantidad de usuarios que han puntuado cada juego
  conteo_puntajes = juegos_de_usuarios_similares.notna().mul(usuarios_similares, axis=0).sum(axis=0)

    # Calculamos el puntaje promedio de cada juego
  puntaje_promedio = suma_puntajes / conteo_puntajes

    # Convertimos el resultado en un DataFrame y lo ordenamos
  item_score = puntaje_promedio.reset_index().rename(columns={'index': 'item_name', 0: 'item_score'}).sort_values(by='item_score', ascending=False)

    # Seleccionamos los 5 mejores juegos
  lista_salida = item_score.head(5)['item_name'].tolist()

  return lista_salida

In [11]:
recomendacion_usuario("")

['Dirty Bomb®',
 'Portal 2',
 'Portal',
 'DARK SOULS™: Prepare To Die™ Edition',
 'Middle-earth™: Shadow of Mordor™']

In [10]:
pd.read_parquet("../Datasets/recomendacion/matriz_normalizada.parquet")

item_name,APB Reloaded,ARK: Survival Evolved,Ace of Spades: Battle Builder,AdVenture Capitalist,Arma 3,Awesomenauts - the 2D moba,Bad Rats: the Rats' Revenge,BattleBlock Theater®,BioShock Infinite,Blacklight: Retribution,...,The Stanley Parable,The Walking Dead,The Walking Dead: Season 2,Tomb Raider,Trove,Undertale,Unturned,Verdun,Warframe,XCOM: Enemy Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
09879655452567,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN
10051997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111222333444555666888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.666667,0.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234567io9872345678765432,-0.2,NaN,NaN,NaN,-0.2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zakbot,NaN,-0.6,-0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zayyntt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.2,NaN,NaN,NaN
zerzang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
